<a href="https://colab.research.google.com/github/averyrair/ChessBAKEN/blob/main/MoveSelector/MoveSelectorBAKEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
#@title Install Dependencies

!pip install chess
#!pip install stockfish
#!apt install stockfish

In [47]:
#@title Import Directories

from google.colab import drive
# For this to work, you need to have the "Chess Bot BAKEN" project shared with
# the SAME email you linked this Colab to, presumably your GitHub email address.
drive.mount('/content/drive', force_remount=1)
chess_dir = '/content/drive/Shareddrives/Chess Bot BAKEN'

Mounted at /content/drive


In [38]:
#@title Import Libraries

import os
import chess
import chess.pgn
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [39]:
#@title Define Custom Board Representation

def customBoardRep(board):
  # From the current board position, this method should output string of ones and zeros of length 8 x 8 x 6 x 2.

  # For now, I'm just passing the FEN.
  return board.fen()

In [ ]:
#@title Import PGN and Export CSV

import chess.pgn
if not os.path.exists(os.path.join(chess_dir, 'moveSelectorDataset.csv')):
  pgn_dir = os.path.join(chess_dir, 'KingBase2019-pgn')
  pgnFiles = [os.path.join(pgn_dir, f) for f in os.listdir(pgn_dir) if f.endswith('.pgn')]

  # Prepare CSV to export dataset.
  with open(os.path.join(chess_dir, 'moveSelectorDataset.csv'), 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    #csvwriter.writeheader(['Label', 'Input'])

    # Read every PGN file in the directory.
    iterator = 0
    for pgnFile in pgnFiles:
      print(pgnFile)
      pgn = open(pgnFile)

      # Read every game in the PGN file.
      while True:
        game = chess.pgn.read_game(pgn)
        if game is None: break
        iterator += 1
        if iterator % 500 == 0: print(iterator)
        board = game.board()

        # New data point for every move in the game.
        gameMoves = list(game.mainline_moves())
        for i in range(len(gameMoves)-1):
          board.push(gameMoves[i])
          legalMoves = " "
          for m in list(board.legal_moves):
            legalMoves = legalMoves + str(m) + " "
          #print(gameMoves[i+1], legalMoves, customBoardRep(board))
          csvwriter.writerow([gameMoves[i+1], legalMoves + customBoardRep(board)])

In [5]:
#@title Define Fully-Connected Neural Network

class myFCN(nn.Module):
  def __init__(self, inSize, hiddenSizes, outSize):
    super().__init__()
    self.inSize = inSize
    self.hiddenSize = hiddenSizes
    self.outSize = outSize

    self.lin1 = nn.Linear(inSize, hiddenSizes[0])
    self.lin2 = nn.Linear(hiddenSizes[0], hiddenSizes[1])
    self.lin3 = nn.Linear(hiddenSizes[1], outSize)

    self.bn1 = nn.BatchNorm2d(hiddenSizes[0])
    self.bn2 = nn.BatchNorm2d(hiddenSizes[1])

    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.bn1(self.lin1(x)))
    x = self.relu(self.bn2(self.lin2(x)))
    x = self.lin3(x)
    return x

In [6]:
#@title Initialize Fully-Connected Neural Network

moveSelector = myFCN(inSize=2560, hiddenSizes=[2200, 2200], outSize=1792)
moveSelector

myFCN(
  (lin1): Linear(in_features=2560, out_features=2200, bias=True)
  (lin2): Linear(in_features=2200, out_features=2200, bias=True)
  (lin3): Linear(in_features=2200, out_features=1792, bias=True)
  (bn1): BatchNorm2d(2200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(2200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)

In [7]:
#@title Define Dataset

class ChessDataset(Dataset):
    def __init__(self, chessgame_file, chess_dir, transform=None, target_transform=None):
        self.game_labels = pd.read_csv(chessgame_file)
        self.chess_dir = chess_dir  # This is not used. Everything is in the chessgame file.
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.game_labels)

    def __getitem__(self, idx):
        label = self.game_labels.iloc[idx, 0]
        gameState = self.game_labels.iloc[idx, 1]
        if self.transform:
            gameState = self.transform(gameState)
        if self.target_transform:
            label = self.target_transform(label)
        return gameState, label